In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
import sqlalchemy.dialects
from sqlalchemy.dialects import registry
from sqlalchemy.orm import sessionmaker
import seaborn as sns

In [2]:
# Define and import from CME ftp
file_name = "ftp://ftp.cmegroup.com/pub/settle/nymex_option.csv"

In [3]:
# Create Dataframe
df = pd.read_csv(file_name, index_col=False)

# Select Columns
df.drop (["CONTRACT DAY", "CONTRACT", "OPEN", "HIGH", "PRODUCT DESCRIPTION", "HIGH AB INDICATOR", 
            "LOW", "LOW AB INDICATOR", "LAST", "LAST AB INDICATOR"
            ], axis = 1, inplace = True)

df_opts = pd.DataFrame (df,columns = ["PRODUCT SYMBOL", "CONTRACT MONTH", "CONTRACT YEAR", "PUT/CALL", "STRIKE", "SETTLE", "PT CHG", "EST. VOL", "PRIOR SETTLE", "PRIOR VOL", "PRIOR INT", "TRADEDATE"])

# Remove display delimiters
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)
pd.set_option("display.max_colwidth", None)

# Convert NaNs to numeric format
df_opts["EST. VOL"] = pd.to_numeric(df_opts["EST. VOL"], errors="coerce")
df_opts["PT CHG"] = pd.to_numeric(df_opts["PT CHG"], errors="coerce")
df_opts["PRIOR VOL"] = pd.to_numeric(df_opts["PRIOR VOL"], errors="coerce")
df_opts["PRIOR INT"] = pd.to_numeric(df_opts["PRIOR INT"], errors="coerce")

# Drop Null numerics
df_opts = df_opts.dropna(subset=["EST. VOL"])
df_opts = df_opts.dropna(subset=["PRIOR VOL"])
df_opts = df_opts.dropna(subset=["PRIOR INT"])

# Convert float to int
df_opts["EST. VOL"] = df_opts["EST. VOL"].astype(int)
df_opts["PRIOR VOL"] = df_opts["PRIOR VOL"].astype(int)
df_opts["PRIOR INT"] = df_opts["PRIOR INT"].astype(int)

# Parse product symbol to LN (European Gas Options)
df1 = df_opts[df_opts["PRODUCT SYMBOL"].str.contains("LN")]

# Remove empty estimated volume rows
df1.loc[df1["EST. VOL"] > 0]

# Return estimated volume in descending max to min order
df1.sort_values (by = ["EST. VOL"], inplace = True, ascending = False)
df1.head()

,PRODUCT SYMBOL,CONTRACT MONTH,CONTRACT YEAR,PUT/CALL,STRIKE,SETTLE,PT CHG,EST. VOL,PRIOR SETTLE,PRIOR VOL,PRIOR INT,TRADEDATE
21753,LN,4,2021,C,4.00,0.0347,0.0030,9200,0.0317,959,23472,11/23/2020
20601,LN,12,2020,P,2.70,0.0393,-0.0575,4708,0.0968,2029,7097,11/23/2020
20406,LN,12,2020,C,2.80,0.0088,-0.0072,4479,0.0160,5633,7217,11/23/2020
20403,LN,12,2020,C,2.75,0.0205,-0.0060,4146,0.0265,2211,9887,11/23/2020
20751,LN,1,2021,C,4.00,0.0310,-0.0032,3278,0.0342,3176,13240,11/23/2020


In [14]:
#Total Options Volume
Total_Volume = df1 ["EST. VOL"].sum()
Total_Prior = df1 ["PRIOR VOL"].sum()
Total_OI = df1 ["PRIOR INT"].sum()
print ("Yesterday's total options volume was", Total_Volume,"."" Prior volume was", Total_Prior, "while open interest stands at", Total_OI,".")

Yesterday's total options volume was 73623 . Prior volume was 108473 while open interest stands at 1159955 .


In [5]:
# Total Call Volume